In [1]:
import utils.datareader as datareader
import utils.visualizer as visualizer
from models.transformer import *

In [7]:
datafolder = 'C://Users//marku//Desktop//krafthack_files'

targets = [f'Bolt_{n+1}_Tensile' for n in range(6)]
learning_rate = 3e-3 # optimizer
learning_rate_annealing = 1e-5 # how much the learning rate is annealed every episode
episodes = 5000 # how often it is iterated over the entire dataset
inputlength = 100
batchsize = 50

num_layers=3
dropout=0. 
no_heads=4
hidden_size=no_heads*2*5


Read and scale datasets:

In [3]:
df, dfT = datareader.df_from_folder(datafolder)
df, dfT, scaler = datareader.normalize_input(df, dfT)

features = list(dfT.columns) # find features

Remove overhead data, find the nan timestamps and mask nans:

In [4]:
# remove overhead:
df = df[targets+features] 
# find nan timestamps:
nans = np.array(df[targets].isna().product(1)) * np.arange(df.shape[0]) 
nans = nans[nans != 0]
# mask nans:
df = df.fillna(0)
dfT = dfT.fillna(0)

Set up model:

In [9]:
net = Network(df.shape[1], len(targets), hidden_size=hidden_size, num_layers=num_layers, dropout=dropout, no_heads=no_heads, maxlen=inputlength)
net.train() # starting in training mode
optim = torch.optim.Rprop(net.parameters(), lr=learning_rate)
loss_fn = nn.MSELoss()

In [10]:
plot_episode = 100 # which episodes to plot
losshistory = []
for e in range(episodes):
    # zero gradients out:
    net.zero_grad() 
    # sample a batch:
    x, y = sample(df, nans, targets, inputlength, batchsize, device=net.device)
    # predict:
    y_pred = net(x)      
    # calculate losses:
    loss = loss_fn(y_pred, y)
    # backpropagation:
    loss.backward()
    optim.step()       
    # save losses:
    losshistory.append(loss.detach().cpu().numpy())  
    print(f'episode {e+1}: {losshistory[-1]}')
    # anneal learning rate:
    for g in optim.param_groups:
        g['lr'] = g['lr'] * (1 - learning_rate_annealing)



episode 1: 0.3590555787086487
episode 2: 13.63988971710205
episode 3: 2.7828145027160645
episode 4: 3.541452646255493
episode 5: 10.948371887207031
episode 6: 0.48356160521507263
episode 7: 1.847572922706604
episode 8: 0.4560913145542145
episode 9: 1.326471209526062
episode 10: 0.506493330001831
episode 11: 0.05032346770167351
episode 12: 0.11772862821817398
episode 13: 0.03836246579885483
episode 14: 0.04962113872170448
episode 15: 0.04805099591612816
episode 16: 0.02248874120414257
episode 17: 0.028128981590270996
episode 18: 0.022768229246139526
episode 19: 0.012055747210979462
episode 20: 0.013921265490353107
episode 21: 0.011746985837817192
episode 22: 0.008911716751754284
episode 23: 0.009944911114871502
episode 24: 0.00877051055431366
episode 25: 0.00857184361666441
episode 26: 0.007870589382946491
episode 27: 0.008783848956227303
episode 28: 0.006793079432100058
episode 29: 0.010514836758375168
episode 30: 0.008089403621852398
episode 31: 0.007827186957001686
episode 32: 0.0081

KeyboardInterrupt: 